# Preparation

In [0]:
import os
import time
from tqdm import tqdm, trange
import cv2
import matplotlib.pyplot as plt
import numpy as np

Clone [darkflow](https://github.com/thtrieu/darkflow.git) (darknet translation to tensorflow)

In [0]:
os.chdir('/content/')
!rm -rf '/content/darkflow'
!git clone https://github.com/thtrieu/darkflow.git
!pip install Cython 

os.chdir('/content/darkflow')

Install darkflow

In [0]:
!pwd

!pip install -e .

Weights and configurations from pretrained models are downloaded from [original YOLO cv2 implementation](https://pjreddie.com/darknet/yolov2/) (in darknet)

In [0]:
# load training weights

newpath = '/content/darkflow/bin'
if not os.path.exists(newpath):
  os.makedirs(newpath)

!wget https://pjreddie.com/media/files/yolov2.weights -O /content/darkflow/bin/yolov2.weights
!wget https://github.com/pjreddie/darknet/raw/master/cfg/yolov2.cfg -O /content/darkflow/cfg/yolov2.cfg

In [0]:
from darkflow.net.build import TFNet

# Train new model on JAAD dataset

##  Configure network

Define labels that should be detected

In [0]:
labels = ['person']

Edit model cfg file accordingly

1.   Make copy basis model file
2.   Change model parameters in last and second-to-last layer according to number of labels





In [0]:
!cp /content/darkflow/cfg/yolov2.cfg /content/darkflow/cfg/yolo-jaad.cfg

In [0]:
with open('/content/darkflow/cfg/yolo-jaad.cfg', 'r') as readfile:
  ## read file
  data = readfile.readlines()
  
## get data
classes = len(labels)
num = int(data[245].split('=')[1])
#print('num: {}'.format(num))
filters = num * (classes + 5)
#print('filters: {}'.format(filters))

## modify file
data[236] = '='.join([data[236].split('=')[0], str(filters)]) + '\n'
#print(data[236])
data[243] = '='.join([data[243].split('=')[0], str(classes)]) + '\n'
#print(data[243])

## write file
with open('/content/darkflow/cfg/yolo-jaad.cfg', 'w') as writefile:
  writefile.writelines(data)

Edit labels.txt file according to specification

In [0]:
with open('/content/darkflow/labels.txt', 'w') as file:
  file.write('\n'.join(labels))

**options** to **start** training **initially** (starting with weights from yolo v2)

In [0]:
options = {"model": "/content/darkflow/cfg/yolo-jaad.cfg", 
           "load": "/content/darkflow/bin/yolov2.weights",
           "batch": 8,
           "epoch": 100,
           "gpu": 1.0,
           "train": True,
           "annotation": "/content/JAAD/train/vbb",
           "dataset": "/content/JAAD/train/vid"}

**options** to start training by **loading from checkpoint**

In [0]:
options = {"model": "/content/darkflow/cfg/yolo-jaad.cfg",
           "load": -1,
           "batch": 8,
           "lr": 1e-6,
           "epoch": 100,
           "gpu": 1.0,
           "train": True,
           "annotation": "/content/JAAD/train/vbb",
           "dataset": "/content/JAAD/train/vid"}

get checkpoint data from Google Drive

In [0]:
checkpoint = 26500 # change checkpoint number here according to the one that should be downloaded

In [0]:
!pip install PyDrive

import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

download = drive.CreateFile({'id': '1luMwEyJcoUFqnkG24oRFvQJulfRd1fM6'})
download.GetContentFile('/content/ckpt{}.zip'.format(checkpoint))

In [0]:
# change checkpoint number here as well
!unzip /content/ckpt26500.zip -d /

In [0]:
ckptpath = '/content/darkflow/ckpt'
if not os.path.exists(ckptpath):
  os.makedirs(ckptpath)

import shutil as sh
ckptroot = '/content/ckpt_{}'.format(checkpoint)
ckpts = os.listdir(ckptroot)

for file in ckpts:
  sh.copyfile(os.path.join(ckptroot, file), os.path.join(ckptpath, file))

## Make model

In [0]:
newtfnet = TFNet(options)

In [0]:
# this step is only necessary when loading weights from checkpoint
newtfnet.load_from_ckpt()

## Start training

In [0]:
newtfnet.train()

#Prediction

**prediction on JAAD test set**

Load Test Data

In [0]:
download = drive.CreateFile({'id': '1_vA0lws_meZvYbNK_hTKtQPw1NXUmH4_'})
download.GetContentFile('/content/testData.zip')

In [0]:
!unzip /content/testData.zip -d /

In [0]:
# reset folder for prediction
!rm -rf /content/JAAD/test/predict_yoloJAAD
!rm -rf /content/JAAD/test/predict_yoloV2

Prediction Function

In [0]:
jaadTestPath = '/content/JAAD/test/vid'
jaadTestImages = os.listdir(jaadTestPath)
jaadTestImages.sort()

def make_prediction(jaadPredictPath, jaadTestImages):

  numImages = len(jaadTestImages)

  strtTime = time.time()

  for image in tqdm(jaadTestImages, desc='image'):
    imageName = os.path.splitext(image)[0]
    imagePath = os.path.join(jaadTestPath, image)

    imageRead = cv2.imread(imagePath)

    result = tfnet.return_predict(imageRead)

    predictionFile = '.'.join([imageName, 'txt'])
    predictionPath = os.path.join(jaadPredictPath, predictionFile)

    tqdm.write('start writing txt file')
    with open(predictionPath, 'w') as wfile:

      for n,value in enumerate(result):
        left = result[n]['topleft']['x']
        top = result[n]['topleft']['y']
        right = result[n]['bottomright']['x']
        bottom = result[n]['bottomright']['y']
        confidence = result[n]['confidence']
        label = result[n]['label']
        if label == 'person':
          wfile.write(' '.join([label, str(confidence), str(left), str(top), str(right), str(bottom)]) + '\n')

  endTime = time.time()
  procTime = endTime - strtTime
  avProcTime = procTime / numImages
  print('Number of images processed: {}'.format(numImages))
  print('Total time to process all images: {}'.format(procTime))
  print('Average processing time per image: {}'.format(avProcTime))

## Testing the YOLO detector trained on the JAAD dataset

### Configure network

In [0]:
os.chdir('/content/darkflow')

In [0]:
options = {
    'model': 'cfg/yolo-jaad.cfg',
    'load': -1,
    'threshold': 0.3,
    'gpu' : 1.0
          }

In [0]:
labels = ['person']
with open('/content/darkflow/labels.txt', 'w') as file:
  file.write('\n'.join(labels))

In [0]:
jaadPredictPath = '/content/JAAD/test/predict_yoloJAAD'
if not os.path.exists(jaadPredictPath):
  os.makedirs(jaadPredictPath)

### Make model

In [0]:
tfnet = TFNet(options)

### Make predictions

In [0]:
make_prediction(jaadPredictPath, jaadTestImages)

## Testing the YOLOv2 detector pretrained on the COCO dataset

### Configure network

In [0]:
options = {
    'model': 'cfg/yolov2.cfg',
    'load': 'bin/yolov2.weights',
    'threshold': 0.3,
    'gpu' : 1.0
          }

In [0]:
!cp /content/darkflow/cfg/coco.names /content/darkflow/labels.txt

In [0]:
jaadPredictPath = '/content/JAAD/test/predict_yoloV2'
if not os.path.exists(jaadPredictPath):
  os.makedirs(jaadPredictPath)

### Make model

In [0]:
tfnet = TFNet(options)

### Make predictions

In [0]:
make_prediction(jaadPredictPath, jaadTestImages)

## Upload data on Drive

In [0]:
os.chdir('/content/')

!zip -r testData.zip /content/JAAD/test

Output hidden; open in https://colab.research.google.com to view.

In [0]:
# Install the PyDrive wrapper & import libraries.
# This only needs to be done once in a notebook.
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once in a notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

uploaded = drive.CreateFile({'title': 'testData.zip'})
uploaded.SetContentFile('/content/testData.zip')
uploaded.Upload()
print('Uploaded file with ID {}'.format(uploaded.get('id')))